In [1]:
import re
import numpy as np
import pandas as pd

In [2]:
with open('../input/day13.txt', 'r') as f:
    lines = f.read().splitlines()

lines[:10]

['Button A: X+83, Y+59',
 'Button B: X+34, Y+99',
 'Prize: X=4485, Y=8127',
 '',
 'Button A: X+13, Y+33',
 'Button B: X+57, Y+19',
 'Prize: X=13101, Y=9731',
 '',
 'Button A: X+18, Y+60',
 'Button B: X+83, Y+58']

In [3]:
def get_base_df():
    dfs = []
    for n in np.arange(1, 101):
        dfs.append(pd.DataFrame((np.ones(100) * n, np.arange(1,101))).T)
    combos = pd.concat(dfs).rename(columns={0:'A', 1:'B'}).assign(tokens = lambda x: 3*x['A'] + x['B'])
    return combos

In [4]:
def get_combinations(A_x, A_y, B_x, B_y, prize_x, prize_y):
    df = (
        get_base_df()
        .assign(
            value_x = lambda x: x.A*A_x + x.B*B_x,
            value_y = lambda x: x.A*A_y + x.B*B_y,
        )
    )

    return df

In [5]:
tokens = 0
for machine in range(len(lines))[::4]:
    A_x, A_y = [int(x) for x in re.findall('\d+', lines[machine])]
    B_x, B_y = [int(x) for x in re.findall('\d+', lines[machine+1])]
    prize_x, prize_y = [int(x) for x in re.findall('\d+', lines[machine+2])]

    df = get_combinations(A_x, A_y, B_x, B_y, prize_x, prize_y)
    tokens += max(0, df.loc[
        ((df.value_x == prize_x)
        & (df.value_y == prize_y))
    ].min().tokens)

tokens

np.float64(36838.0)

In [126]:
#p2; let's do this properly
tokens = 0
for machine in range(len(lines))[::4]:
    p2 = 10000000000000
    A_x, A_y = [int(x) for x in re.findall('\d+', lines[machine])]
    B_x, B_y = [int(x) for x in re.findall('\d+', lines[machine+1])]
    prize_x, prize_y = [int(x)+p2 for x in re.findall('\d+', lines[machine+2])]
    
    lhs = np.array([[A_x, B_x], [A_y, B_y]])
    rhs = np.array([prize_x, prize_y])


    solution = np.rint(np.linalg.solve(lhs, rhs))
    if np.all(np.column_stack(([A_x, A_y], [B_x, B_y])) @ solution == rhs):
            min_token = int(3 * solution[0] + solution[1])
            tokens += max(0, min_token)

tokens

83029436920891